# Merging Thousands of ```.txt()``` Files into a ```.parquet``` File
### Merging all InfoUSA files and saving them as a ```.parquet``` file

### Import statements

In [2]:
import pyarrow
import re
import os
import pandas as pd

### This will stop warnings from showing up

In [3]:
import warnings
warnings.filterwarnings('ignore')

### Merging 00000-25000
We initially attempted to merge all files in one go and save the combined file as a ```.csv()``` file. Although we were able to do this, reading in the combined ```.csv()``` file would not work- our kernel crashed every time we attempted it, even using 38 CPU cores and 208GB of RAM. 

Therefore, we decided to use pyrarrow as an engine when reading in each of the ```.txt``` files and save the combined file as a ```.parquet()``` file. Pyarrow allows for multithreading, leading to a more efficient use of CPU cores, and Parquet files are optimized for storing large datasets. This is because Parquet files are column-based while CSV files are row-based, meaning it can only focus on the data relevant to the user's query. Like this, it only loads into memory the columns of the data that are being used. Parquet files can also be read in and exported using pandas, which is an added bonus when working with pandas dataframes.

Still, we ran into memory issues merging all files in one go- so we broke up our data into four sections, merged the files in each of these sections, dropped columns not relevant to our work, then combined the four sections for one totally combined dataframe.

Here, we are merging zip codes in the range 00000 - 25000. The regular expression sifts through the list of all files in the InfoUSA folder and creates a list of only the files with zip codes in that range.

In [13]:
os.chdir("/hpc/group/codeplus22-vis/infousa_copy/2020")
directory = os.listdir("/hpc/group/codeplus22-vis/celine_data/2020")

r = re.compile("Household_Ethnicity_zip_(0{4}[0-9]|0{3}[1-9][0-9]|00[1-9][0-9]{2}|0[1-9][0-9]{3}|1[0-9]{4}|2[0-4][0-9]{3}|25000)_year_2020.txt")

newlist = list(filter(r.match, directory)) # Read Note
# print(newlist)
len(newlist)

8706

Then, we use the pandas ```.concat()``` function to concatenate every file in the list of files created above.

In [20]:
%%time
#combine all files in the list
zip_00_25 = pd.concat([pd.read_csv(f, sep = '\t', 
                                   dtype = {'census_state_2010': 'str', 'census_county_2010': 'str'}) for f in newlist])
zip_00_25.head()



CPU times: user 3min 35s, sys: 22.2 s, total: 3min 57s
Wall time: 4min 22s


FamilyID location_type  primary_family_ind HouseholdStatus  \
0  101133133             S                   1               F   
1  105579259             U                   0               F   
2  282131063             S                   1               F   
3  340638960             S                   1               F   
4  398667313             S                   1               F   

   tradeline_count head_hh_age_code  length_of_residence  ChildrenHHCount  \
0                0                K                    2                0   
1                0                H                    9                0   
2                0                C                    3                1   
3                0                L                   27                0   
4                0                H                   33                1   

   children_ind AddressType  ...  GE_CENSUS_LEVEL_2010  middle_name  \
0             0           S  ...                     P          NaN   
1             0           P  ...                     Z            M   
2             1           S  ...                     P            S   
3             0           S  ...                     P            S   
4             1           S  ...                     P            J   

   last_name_1  Ethnicity_Code_1  first_name_2  last_name_2  Ethnicity_Code_2  \
0         TODD                E5       DEBORAH         TODD                E5   
1         WOOD                E5         WENDY         WOOD                E5   
2        BRADY                IE        THOMAS        BRADY                IE   
3        MAGEE                IE          RUTH        MAGEE                IE   
4        FINAN                IE           NaN          NaN               NaN   

   first_name_3  last_name_3  Ethnicity_Code_3  
0       KRISTIN         TODD                E5  
1           NaN          NaN               NaN  
2          NITA        BRADY                IE  
3           NaN          NaN               NaN  
4           NaN          NaN               NaN  

[5 rows x 54 columns]

Filter for only the columns relevant to our future work:

In [21]:
%%time
zip_00_25['county_fips'] = zip_00_25['census_state_2010'] + zip_00_25['census_county_2010']

zip_00_25 = zip_00_25[['ZIP', 'county_fips', 'STATE', 'ChildrenHHCount', 'children_ind', 
                       'head_hh_age_code', 'GE_LATITUDE_2010', 'GE_LONGITUDE_2010']]

zip_00_25

CPU times: user 20.9 s, sys: 28 s, total: 48.9 s
Wall time: 49 s


ZIP county_fips STATE  ChildrenHHCount  children_ind head_hh_age_code  \
0     18833       42113    PA                0             0                K   
1     18833       42015    PA                0             0                H   
2     18833       42015    PA                1             1                C   
3     18833       42015    PA                0             0                L   
4     18833       42015    PA                1             1                H   
...     ...         ...   ...              ...           ...              ...   
1528  18322       42089    PA                0             0                G   
1529  18322       42089    PA                0             0                E   
1530  18322       42089    PA                0             0                E   
1531  18322       42089    PA                0             0                B   
1532  18322       42089    PA                0             0                E   

      GE_LATITUDE_2010  GE_LONGITUDE_2010  
0            41.546738         -76.540436  
1            41.590800         -76.424200  
2            41.600392         -76.441724  
3            41.592483         -76.437832  
4            41.566196         -76.347977  
...                ...                ...  
1528         40.927534         -75.401987  
1529         40.926938         -75.375047  
1530         40.912001         -75.377969  
1531         40.918671         -75.388654  
1532         40.926839         -75.393408  

[43593386 rows x 8 columns]

Filter for only zipcodes not in Alaska, Hawaii, Puerto Rico, and the Virgin Islands, as there are no storage tanks in the AST dataset from these areas:

In [22]:
%%time
zip_00_25 = zip_00_25[((zip_00_25['STATE'] != 'AK') & (zip_00_25['STATE'] != 'HI') & (zip_00_25['STATE'] != 'PR') & (zip_00_25['STATE'] != 'VI'))]

zip_00_25

CPU times: user 10 s, sys: 359 ms, total: 10.4 s
Wall time: 10.3 s


ZIP county_fips STATE  ChildrenHHCount  children_ind head_hh_age_code  \
0     18833       42113    PA                0             0                K   
1     18833       42015    PA                0             0                H   
2     18833       42015    PA                1             1                C   
3     18833       42015    PA                0             0                L   
4     18833       42015    PA                1             1                H   
...     ...         ...   ...              ...           ...              ...   
1528  18322       42089    PA                0             0                G   
1529  18322       42089    PA                0             0                E   
1530  18322       42089    PA                0             0                E   
1531  18322       42089    PA                0             0                B   
1532  18322       42089    PA                0             0                E   

      GE_LATITUDE_2010  GE_LONGITUDE_2010  
0            41.546738         -76.540436  
1            41.590800         -76.424200  
2            41.600392         -76.441724  
3            41.592483         -76.437832  
4            41.566196         -76.347977  
...                ...                ...  
1528         40.927534         -75.401987  
1529         40.926938         -75.375047  
1530         40.912001         -75.377969  
1531         40.918671         -75.388654  
1532         40.926839         -75.393408  

[43593386 rows x 8 columns]

Write out the dataframe as a ```.parquet()``` file:

In [23]:
%%time
zip_00_25.to_parquet('/hpc/group/codeplus22-vis/celine_data/zip_00_25.parquet')

CPU times: user 12 s, sys: 779 ms, total: 12.8 s
Wall time: 13.2 s


### Merging 25000-50000
We use the same steps as Merging 00000 - 25000 section.

In [4]:
os.chdir("/hpc/group/codeplus22-vis/infousa_copy/2020")
directory = os.listdir("/hpc/group/codeplus22-vis/celine_data/2020")
r = re.compile("Household_Ethnicity_zip_(2500[0-9]|250[1-9][0-9]|25[1-9][0-9]{2}|2[6-9][0-9]{3}|[34][0-9]{4}|50000)_year_2020.txt")
newlist = list(filter(r.match, directory)) # Read Note
# print(newlist)
len(newlist)

10758

In [5]:
zip_25_50 = pd.concat([pd.read_csv(f, sep = '\t', 
                                   dtype = {'census_state_2010': 'str', 'census_county_2010': 'str'}) for f in newlist])
zip_25_50.head()

FamilyID location_type  primary_family_ind HouseholdStatus  \
0     204505             S                   1               F   
1  207105250             S                   1               F   
2  210892206             S                   1               F   
3  210892421             S                   1               F   
4  210920662             U                   1               F   

   tradeline_count head_hh_age_code  length_of_residence  ChildrenHHCount  \
0                2                G                   19                0   
1                0                H                   11                1   
2                0                F                   11                5   
3                0                H                   21                1   
4                0                I                   26                0   

   children_ind AddressType  ...  GE_CENSUS_LEVEL_2010  middle_name  \
0             0           S  ...                     P            R   
1             1           S  ...                     P            M   
2             1           S  ...                     P            I   
3             1           S  ...                     P            R   
4             0           P  ...                     Z            K   

   last_name_1  Ethnicity_Code_1  first_name_2  last_name_2  Ethnicity_Code_2  \
0        DEFEE                E5    JACQUELINE        DEFEE                E5   
1        SLACK                B5        YVONNE        SLACK                S3   
2         ROWE                E5       REBEKAH         ROWE                E5   
3      ROEHLER                DE         ANITA      ROEHLER                DE   
4         ROBY                E5       MELISSA         ROBY                E5   

   first_name_3  last_name_3  Ethnicity_Code_3  
0           NaN          NaN               NaN  
1        ANDREA        SLACK                S3  
2         MEGAN         ROWE                E5  
3       MIRANDA      ROEHLER                DE  
4           NaN          NaN               NaN  

[5 rows x 54 columns]

In [6]:
%%time
zip_25_50['county_fips'] = zip_25_50['census_state_2010'] + zip_25_50['census_county_2010']
zip_25_50 = zip_25_50[['ZIP', 'county_fips', 'STATE', 'ChildrenHHCount', 'children_ind', 'head_hh_age_code', 'GE_LATITUDE_2010', 'GE_LONGITUDE_2010']]

CPU times: user 29.9 s, sys: 25.1 s, total: 55 s
Wall time: 55.1 s


In [8]:
%%time
zip_25_50 = zip_25_50[((zip_25_50['STATE'] != 'AK') & (zip_25_50['STATE'] != 'HI') & (zip_25_50['STATE'] != 'PR') & (zip_25_50['STATE'] != 'VI'))]
zip_25_50

CPU times: user 14 s, sys: 581 ms, total: 14.6 s
Wall time: 14.5 s


ZIP county_fips STATE  ChildrenHHCount  children_ind head_hh_age_code  \
0     43345       39159    OH                0             0                G   
1     43345       39091    OH                1             1                H   
2     43345       39065    OH                5             1                F   
3     43345       39091    OH                1             1                H   
4     43345       39065    OH                0             0                I   
...     ...         ...   ...              ...           ...              ...   
3717  39474       28065    MS                0             0                F   
3718  39474       28065    MS                0             0                B   
3719  39474       28065    MS                0             0                F   
3720  39474       28065    MS                0             0                F   
3721  39474       28065    MS                0             0                F   

      GE_LATITUDE_2010  GE_LONGITUDE_2010  
0            40.501032         -83.518956  
1            40.512661         -83.570311  
2            40.524095         -83.574434  
3            40.491840         -83.523492  
4            40.515700         -83.573400  
...                ...                ...  
3717         31.613300         -89.856400  
3718         31.596286         -89.866823  
3719         31.662703         -89.789536  
3720         31.610200         -89.871200  
3721         31.594625         -89.872358  

[60400598 rows x 8 columns]

In [9]:
%%time
zip_25_50.to_parquet('/hpc/group/codeplus22-vis/celine_data/zip_25_50.parquet')

CPU times: user 16.7 s, sys: 1.07 s, total: 17.8 s
Wall time: 18.2 s


### Merging 50000-75000
We use the same steps as Merging 00000 - 25000 section.

In [10]:
os.chdir("/hpc/group/codeplus22-vis/infousa_copy/2020")
directory = os.listdir("/hpc/group/codeplus22-vis/celine_data/2020")
r = re.compile("Household_Ethnicity_zip_(5000[0-9]|500[1-9][0-9]|50[1-9][0-9]{2}|5[1-9][0-9]{3}|6[0-9]{4}|7[0-4][0-9]{3}|75000)_year_2020.txt")
newlist = list(filter(r.match, directory)) # Read Note
# print(newlist)
len(newlist)

9917

In [11]:
%%time
#combine all files in the list
zip_50_75 = pd.concat([pd.read_csv(f, sep = '\t', dtype = {'census_state_2010': 'str', 'census_county_2010': 'str'}) for f in newlist])
zip_50_75.head()

CPU times: user 4min 41s, sys: 13.4 s, total: 4min 54s
Wall time: 9min 19s


FamilyID location_type  primary_family_ind HouseholdStatus  \
0   52749969             S                   1               F   
1  175470537             S                   0               F   
2  175525274             S                   1               F   
3  175566359             S                   1               F   
4  175566362             S                   1               F   

   tradeline_count head_hh_age_code  length_of_residence  ChildrenHHCount  \
0                0                L                   11                0   
1                0                L                   19                1   
2                2                M                   18                0   
3                0                M                   29                0   
4                3                H                   17                2   

   children_ind AddressType  ...  GE_CENSUS_LEVEL_2010  middle_name  \
0             0           S  ...                     P            A   
1             1           S  ...                     P            R   
2             0           S  ...                     P          NaN   
3             1           S  ...                     P            A   
4             1           S  ...                     P          NaN   

   last_name_1  Ethnicity_Code_1  first_name_2  last_name_2  Ethnicity_Code_2  \
0       KNIGHT                W4        SHARON       KNIGHT                W4   
1     HAZLETON                E5           NaN          NaN               NaN   
2      ASHFORD                E5        WALTER      ASHFORD                E5   
3     BARTLETT                E5         LINDA     BARTLETT                E5   
4        BOGGS                E5         MARIE        BOGGS                E5   

   first_name_3  last_name_3  Ethnicity_Code_3  
0        SHERRY       KNIGHT                W4  
1           NaN          NaN               NaN  
2         JULIE      ASHFORD                E5  
3           NaN          NaN               NaN  
4        VIOLET        BOGGS                E5  

[5 rows x 54 columns]

In [12]:
%%time
zip_50_75['county_fips'] = zip_50_75['census_state_2010'] + zip_50_75['census_county_2010']
zip_50_75 = zip_50_75[['ZIP', 'county_fips', 'STATE', 'ChildrenHHCount', 'children_ind', 'head_hh_age_code', 'GE_LATITUDE_2010', 'GE_LONGITUDE_2010']]

CPU times: user 15.4 s, sys: 12.1 s, total: 27.5 s
Wall time: 27.6 s


In [13]:
%%time
zip_50_75 = zip_50_75[((zip_50_75['STATE'] != 'AK') & (zip_50_75['STATE'] != 'HI') & (zip_50_75['STATE'] != 'PR') & (zip_50_75['STATE'] != 'VI'))]
zip_50_75

CPU times: user 7.45 s, sys: 312 ms, total: 7.76 s
Wall time: 7.74 s


ZIP county_fips STATE  ChildrenHHCount  children_ind head_hh_age_code  \
0    74869       40081    OK                0             0                L   
1    74869       40081    OK                1             1                L   
2    74869       40081    OK                0             0                M   
3    74869       40081    OK                0             1                M   
4    74869       40081    OK                2             1                H   
..     ...         ...   ...              ...           ...              ...   
651  53801       55043    WI                0             0                C   
652  53801       55043    WI                0             0                A   
653  53801       55043    WI                0             0                J   
654  53801       55043    WI                0             0                G   
655  53801       55043    WI                0             0                E   

     GE_LATITUDE_2010  GE_LONGITUDE_2010  
0           35.607354         -96.824352  
1           35.548371         -96.727254  
2           35.571250         -96.696193  
3           35.568520         -96.757917  
4           35.607481         -96.761271  
..                ...                ...  
651         42.920500         -91.072000  
652         42.904946         -91.104199  
653         42.920500         -91.072000  
654         42.920500         -91.072000  
655         42.895001         -91.091589  

[30537160 rows x 8 columns]

In [14]:
%%time
zip_50_75.to_parquet('/hpc/group/codeplus22-vis/celine_data/zip_50_75.parquet')

CPU times: user 8.77 s, sys: 423 ms, total: 9.19 s
Wall time: 9.46 s


### Merging 75000-99000
We use the same steps as Merging 00000 - 25000 section.

In [15]:
os.chdir("/hpc/group/codeplus22-vis/infousa_copy/2020")
directory = os.listdir("/hpc/group/codeplus22-vis/celine_data/2020")
r = re.compile("Household_Ethnicity_zip_(7500[0-9]|750[1-9][0-9]|75[1-9][0-9]{2}|7[6-9][0-9]{3}|[89][0-9]{4})_year_2020.txt")
newlist = list(filter(r.match, directory)) # Read Note
# print(newlist)
len(newlist)

8824

In [16]:
%%time
#combine all files in the list
zip_75_99 = pd.concat([pd.read_csv(f, sep = '\t', dtype = {'census_state_2010': 'str', 'census_county_2010': 'str'}) for f in newlist])
zip_75_99.head()

CPU times: user 6min 58s, sys: 24.3 s, total: 7min 23s
Wall time: 12min 13s


FamilyID location_type  primary_family_ind HouseholdStatus  \
0  25899683             U                   1               F   
1  28681487             U                   1               F   
2  28681665             U                   1               F   
3  28681725             U                   1               F   
4  28681762             U                   1               F   

   tradeline_count head_hh_age_code  length_of_residence  ChildrenHHCount  \
0                0                M                   30                0   
1                0                I                   30                0   
2                0                I                   24                1   
3                0                K                   25                0   
4                0                H                   22                0   

   children_ind AddressType  ...  GE_CENSUS_LEVEL_2010  middle_name  \
0             0           P  ...                     Z            J   
1             0           P  ...                     Z            J   
2             1           P  ...                     Z            K   
3             0           P  ...                     Z            W   
4             0           P  ...                     Z            P   

   last_name_1  Ethnicity_Code_1  first_name_2  last_name_2  Ethnicity_Code_2  \
0         LEMN                00           NaN          NaN               NaN   
1      MASTERS                E5           NaN          NaN               NaN   
2        CHENG                CN          LILY        CHENG                CN   
3         DAWS                E5           NaN          NaN               NaN   
4        RAMOS                H5           NaN          NaN               NaN   

   first_name_3  last_name_3  Ethnicity_Code_3  
0           NaN          NaN               NaN  
1           NaN          NaN               NaN  
2           NaN          NaN               NaN  
3           NaN          NaN               NaN  
4           NaN          NaN               NaN  

[5 rows x 54 columns]

In [17]:
%%time
zip_75_99['county_fips'] = zip_75_99['census_state_2010'] + zip_75_99['census_county_2010']
zip_75_99 = zip_75_99[['ZIP', 'county_fips', 'STATE', 'ChildrenHHCount', 'children_ind', 'head_hh_age_code', 'GE_LATITUDE_2010', 'GE_LONGITUDE_2010']]

CPU times: user 28.5 s, sys: 23.2 s, total: 51.7 s
Wall time: 51.8 s


In [18]:
%%time
zip_75_99 = zip_75_99[((zip_75_99['STATE'] != 'AK') & (zip_75_99['STATE'] != 'HI') & (zip_75_99['STATE'] != 'PR') & (zip_75_99['STATE'] != 'VI'))]

CPU times: user 13.9 s, sys: 682 ms, total: 14.5 s
Wall time: 14.5 s


In [26]:
zip_75_99

ZIP county_fips STATE  ChildrenHHCount  children_ind head_hh_age_code  \
0     85346       04012    AZ                1             1                K   
1     85346       04012    AZ                0             0                G   
2     85346       04012    AZ                0             0                K   
3     85346       04012    AZ                0             0                H   
4     85346       04012    AZ                0             0                K   
...     ...         ...   ...              ...           ...              ...   
3174  92003       06073    CA                0             0                C   
3175  92003       06073    CA                0             0                E   
3176  92003       06073    CA                0             0                G   
3177  92003       06073    CA                0             0                H   
3178  92003       06073    CA                0             0                B   

      GE_LATITUDE_2010  GE_LONGITUDE_2010  
0            33.617100        -114.167400  
1            33.617100        -114.167400  
2            33.617100        -114.167400  
3            33.617100        -114.167400  
4            33.617100        -114.167400  
...                ...                ...  
3174         33.285885        -117.240445  
3175         33.284700        -117.210800  
3176         33.282869        -117.183963  
3177         33.278284        -117.181181  
3178         33.292016        -117.234099  

[56456469 rows x 8 columns]

In [25]:
%%time
zip_75_99.to_parquet('/hpc/group/codeplus22-vis/celine_data/zip_75_99.parquet')

CPU times: user 15.8 s, sys: 893 ms, total: 16.7 s
Wall time: 17.8 s


### Merging 00000 - 50000
Now that we merged each of our four sections, we will merge these four sections with eachother using pandas' ```.append()``` function.

In [27]:
%%time
zip_00_50 = pd.read_parquet('/hpc/group/codeplus22-vis/celine_data/zip_00_25.parquet').append(pd.read_parquet('/hpc/group/codeplus22-vis/celine_data/zip_25_50.parquet'), ignore_index = True)
zip_00_50

CPU times: user 25 s, sys: 6.17 s, total: 31.2 s
Wall time: 17 s


ZIP county_fips STATE  ChildrenHHCount  children_ind  \
0          18833       42113    PA                0             0   
1          18833       42015    PA                0             0   
2          18833       42015    PA                1             1   
3          18833       42015    PA                0             0   
4          18833       42015    PA                1             1   
...          ...         ...   ...              ...           ...   
103993979  39474       28065    MS                0             0   
103993980  39474       28065    MS                0             0   
103993981  39474       28065    MS                0             0   
103993982  39474       28065    MS                0             0   
103993983  39474       28065    MS                0             0   

          head_hh_age_code  GE_LATITUDE_2010  GE_LONGITUDE_2010  
0                        K         41.546738         -76.540436  
1                        H         41.590800         -76.424200  
2                        C         41.600392         -76.441724  
3                        L         41.592483         -76.437832  
4                        H         41.566196         -76.347977  
...                    ...               ...                ...  
103993979                F         31.613300         -89.856400  
103993980                B         31.596286         -89.866823  
103993981                F         31.662703         -89.789536  
103993982                F         31.610200         -89.871200  
103993983                F         31.594625         -89.872358  

[103993984 rows x 8 columns]

In [28]:
%%time
zip_00_50.to_parquet('/hpc/group/codeplus22-vis/celine_data/zip_00_50.parquet')

CPU times: user 27.1 s, sys: 1.32 s, total: 28.5 s
Wall time: 29.2 s


### Merging 50000 - 99000
We use the same steps as Merging 00000 - 50000 section.

In [1]:
%%time
zip_50_99 = pd.read_parquet('/hpc/group/codeplus22-vis/celine_data/zip_50_75.parquet').append(pd.read_parquet('/hpc/group/codeplus22-vis/celine_data/zip_75_99.parquet'), ignore_index = True)
zip_50_99

NameError: name 'pd' is not defined

In [30]:
%%time
zip_50_99.to_parquet('/hpc/group/codeplus22-vis/celine_data/zip_50_99.parquet')

CPU times: user 22.9 s, sys: 1.13 s, total: 24.1 s
Wall time: 24.5 s


### Merging 00-99
We use the same steps as Merging 00000 - 25000 section, but this time merge 00000 - 50000 and 50000 - 99999 to get our final, totally combined dataframe.

In [5]:
%%time
zip_00_99 = pd.read_parquet('/hpc/group/codeplus22-vis/celine_data/zip_00_50.parquet').append(pd.read_parquet('/hpc/group/codeplus22-vis/celine_data/zip_50_99.parquet'), ignore_index = True)
zip_00_99

<timed exec>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


CPU times: user 39.4 s, sys: 10.4 s, total: 49.8 s
Wall time: 26.9 s


ZIP county_fips STATE  ChildrenHHCount  children_ind  \
0          18833       42113    PA                0             0   
1          18833       42015    PA                0             0   
2          18833       42015    PA                1             1   
3          18833       42015    PA                0             0   
4          18833       42015    PA                1             1   
...          ...         ...   ...              ...           ...   
190987608  92003       06073    CA                0             0   
190987609  92003       06073    CA                0             0   
190987610  92003       06073    CA                0             0   
190987611  92003       06073    CA                0             0   
190987612  92003       06073    CA                0             0   

          head_hh_age_code  GE_LATITUDE_2010  GE_LONGITUDE_2010  
0                        K         41.546738         -76.540436  
1                        H         41.590800         -76.424200  
2                        C         41.600392         -76.441724  
3                        L         41.592483         -76.437832  
4                        H         41.566196         -76.347977  
...                    ...               ...                ...  
190987608                C         33.285885        -117.240445  
190987609                E         33.284700        -117.210800  
190987610                G         33.282869        -117.183963  
190987611                H         33.278284        -117.181181  
190987612                B         33.292016        -117.234099  

[190987613 rows x 8 columns]

### Renaming columns
We rename the columns in our dataset for standardization purposes.

In [6]:
zip_00_99.rename(columns = {'ZIP': 'zip', 'census_county_2010': 'county', 'STATE': 'state', 'ChildrenHHCount': 'child_num', 
                           'children_ind': 'has_child', 'head_hh_age_code': 'age_code', 'GE_LATITUDE_2010': 'lat_h_4326', 
                            'GE_LONGITUDE_2010': 'lon_h_4326'}, inplace = True)
zip_00_99

zip county_fips state  child_num  has_child age_code  lat_h_4326  \
0          18833       42113    PA          0          0        K   41.546738   
1          18833       42015    PA          0          0        H   41.590800   
2          18833       42015    PA          1          1        C   41.600392   
3          18833       42015    PA          0          0        L   41.592483   
4          18833       42015    PA          1          1        H   41.566196   
...          ...         ...   ...        ...        ...      ...         ...   
190987608  92003       06073    CA          0          0        C   33.285885   
190987609  92003       06073    CA          0          0        E   33.284700   
190987610  92003       06073    CA          0          0        G   33.282869   
190987611  92003       06073    CA          0          0        H   33.278284   
190987612  92003       06073    CA          0          0        B   33.292016   

           lon_h_4326  
0          -76.540436  
1          -76.424200  
2          -76.441724  
3          -76.437832  
4          -76.347977  
...               ...  
190987608 -117.240445  
190987609 -117.210800  
190987610 -117.183963  
190987611 -117.181181  
190987612 -117.234099  

[190987613 rows x 8 columns]

### Transforming household latitude and longitude coordinates from EPSG 4326 to EPSG 3857
A lot of our visualizations need coordinates in EPSG 3857, however these coordinates are in EPSG 4326. Therefore, we use the pyproj interface, which allows us to use the PROJ coordinate transformation software to transform our EPSG 4326 coordinates to EPSG 3857. This creates two new columns in our original dataset with the transformed coordinates.

In [7]:
from pyproj import Proj, Transformer

In [8]:
# Apply transformation
transform_4326_to_3857 = Transformer.from_crs('epsg:4326', 'epsg:3857')
zip_00_99['lat_h_3857'], zip_00_99['lon_h_3857'] = transform_4326_to_3857.transform(
                                                zip_00_99['lat_h_4326'], zip_00_99['lon_h_4326'])

zip_00_99

zip county_fips state  child_num  has_child age_code  lat_h_4326  \
0          18833       42113    PA          0          0        K   41.546738   
1          18833       42015    PA          0          0        H   41.590800   
2          18833       42015    PA          1          1        C   41.600392   
3          18833       42015    PA          0          0        L   41.592483   
4          18833       42015    PA          1          1        H   41.566196   
...          ...         ...   ...        ...        ...      ...         ...   
190987608  92003       06073    CA          0          0        C   33.285885   
190987609  92003       06073    CA          0          0        E   33.284700   
190987610  92003       06073    CA          0          0        G   33.282869   
190987611  92003       06073    CA          0          0        H   33.278284   
190987612  92003       06073    CA          0          0        B   33.292016   

           lon_h_4326    lat_h_3857    lon_h_3857  
0          -76.540436 -8.520442e+06  5.093323e+06  
1          -76.424200 -8.507503e+06  5.099879e+06  
2          -76.441724 -8.509454e+06  5.101307e+06  
3          -76.437832 -8.509021e+06  5.100129e+06  
4          -76.347977 -8.499018e+06  5.096218e+06  
...               ...           ...           ...  
190987608 -117.240445 -1.305115e+07  3.933312e+06  
190987609 -117.210800 -1.304785e+07  3.933154e+06  
190987610 -117.183963 -1.304486e+07  3.932911e+06  
190987611 -117.181181 -1.304455e+07  3.932300e+06  
190987612 -117.234099 -1.305044e+07  3.934129e+06  

[190987613 rows x 10 columns]

### Exporing final dataframe
Finally, we export this dataframe as a ```.parquet()``` file for further use.

In [9]:
%%time
zip_00_99.to_parquet('/hpc/group/codeplus22-vis/celine_data/zip_00_99.parquet')

CPU times: user 59.5 s, sys: 3.29 s, total: 1min 2s
Wall time: 1min 14s


In [4]:
# df = pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_00_99_final_fixed.parquet')
# df

zip county_fips state  child_num  has_child age_code  lat_h_4326  \
0          18833       42113    PA          0          0        K   41.546738   
1          18833       42015    PA          0          0        H   41.590800   
2          18833       42015    PA          1          1        C   41.600392   
3          18833       42015    PA          0          0        L   41.592483   
4          18833       42015    PA          1          1        H   41.566196   
...          ...         ...   ...        ...        ...      ...         ...   
190987608  92003       06073    CA          0          0        C   33.285885   
190987609  92003       06073    CA          0          0        E   33.284700   
190987610  92003       06073    CA          0          0        G   33.282869   
190987611  92003       06073    CA          0          0        H   33.278284   
190987612  92003       06073    CA          0          0        B   33.292016   

           lon_h_4326    lat_h_3857    lon_h_3857  
0          -76.540436 -8.520442e+06  5.093323e+06  
1          -76.424200 -8.507503e+06  5.099879e+06  
2          -76.441724 -8.509454e+06  5.101307e+06  
3          -76.437832 -8.509021e+06  5.100129e+06  
4          -76.347977 -8.499018e+06  5.096218e+06  
...               ...           ...           ...  
190987608 -117.240445 -1.305115e+07  3.933312e+06  
190987609 -117.210800 -1.304785e+07  3.933154e+06  
190987610 -117.183963 -1.304486e+07  3.932911e+06  
190987611 -117.181181 -1.304455e+07  3.932300e+06  
190987612 -117.234099 -1.305044e+07  3.934129e+06  

[190987613 rows x 10 columns]